In [1]:
import sys
sys.path.append("../")
import matplotlib.pyplot as plt

In [3]:
from data.utils import PermittivityCalculator
from dataGeneration.utils import *

import os
import pandas as pd
import numpy as np
from tqdm import tqdm
import random
import imageio
from PIL import Image

random.seed(0)
np.random.seed(0)

materials = [
    "al2o3",
    "sio2",
]
pc = PermittivityCalculator()

f_name = '../dataGeneration/csv/Au_interpolated_1.csv'
content = pd.read_csv(f_name)

wl = content["wl"].values / 1e3
DATA_ROOT = '../dataGeneration/E1Data/'

lambd = 1e-9*content['wl'].values
e2 = content['er'].values + 1j*content['ei'].values
e3 = 1.00 + 1j*0.0

In [5]:

def helperUtil(data):
    r1, r2, e1, material, counter = data
    tr = {
        "r1": r1*1e-9,
        "r2": r2*1e-9,
    }
    te = {
        'e1': e1,
        'e2': e2,
        'e3': e3,
    }

    _, a_abs = getArea(
        tr, te, lambd,
        write=None,
        f_out=None
    )
    
    fig = plt.figure(figsize=(8, 8))
    plt.plot(lambd, a_abs)
    plt.title(f"r1 : {r1}, r2 : {r2}")
    plt.close()
    fig.canvas.draw()
    image = np.frombuffer(fig.canvas.tostring_rgb(), dtype='uint8')
    image  = image.reshape(fig.canvas.get_width_height()[::-1] + (3,))
    return image

def helper(material, samples=None):

    counter = 0
    
    e1 = np.array([pc.getEps(wl_, element=material, mode="complex") for wl_ in wl])
    r1_min, r1_max = 5, 50
    
    plots = []
    
    for r1 in range(r1_min, r1_max+1):
        for r2 in tqdm(range(r1+1, r1+101)):
            data = (r1, r2, e1, material, counter)
            fig = helperUtil(data)
            plots.append(fig)
            
            counter += 1
            
            if counter==samples:
                break
        if counter>=samples:
            break
    
    imageio.mimsave(f'3/cross_section_variation_{material}.gif', plots, fps=10)

    img = Image.open(f'3/cross_section_variation_{material}.gif')
    img.save(f'3/cross_section_variation_{material}.webp', format='WEBP', save_all=True)

    os.remove(f'3/cross_section_variation_{material}.gif')


for material in materials:
    if not os.path.exists('3'):
        os.mkdir('3')
    helper(material, samples=1000)

 98%|█████████▊| 98/100 [00:08<00:00, 11.04it/s]
